In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

## Part 1 - Data Preprocessing

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
    )

In [ ]:
train_data = train_datagen.flow_from_directory(directory='/content/drive/My Drive/lane1_4_parts', 
                                               target_size=(64, 64), 
                                               class_mode='categorical',
                                               batch_size=64, 
                                               subset='training',
                                               shuffle=True)

Found 6556 images belonging to 4 classes.


In [ ]:
validation_data = train_datagen.flow_from_directory(directory='/content/drive/My Drive/lane1_4_parts', 
                                                    target_size=(64, 64),
                                                    batch_size=64,
                                                    class_mode='categorical',
                                                    subset='validation',
                                                    shuffle=True)

Found 1638 images belonging to 4 classes.


In [ ]:
train_data.labels

array([0, 0, 0, ..., 3, 3, 3], dtype=int32)

## Part 2 - Building the CNN

In [ ]:
cnn3 = tf.keras.models.Sequential()

In [ ]:
cnn3.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=[64, 64, 3]))
#cnn3.add(tf.keras.layers.Dropout(0.5))
cnn3.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

In [ ]:
cnn3.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, padding='same', activation='relu',))
cnn3.add(tf.keras.layers.BatchNormalization())
#cnn3.add(tf.keras.layers.Dropout(0.5))
cnn3.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

In [ ]:
cnn3.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu',))
cnn3.add(tf.keras.layers.BatchNormalization())
cnn3.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
cnn3.add(tf.keras.layers.Dropout(0.5))

In [ ]:
cnn3.add(tf.keras.layers.Conv2D(filters=80, kernel_size=3, padding='same', activation='relu',))
cnn3.add(tf.keras.layers.BatchNormalization())
cnn3.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
cnn3.add(tf.keras.layers.Dropout(0.5))

In [ ]:
cnn3.add(tf.keras.layers.Flatten())

In [ ]:
cnn3.add(tf.keras.layers.Dense(units=64, activation='relu'))

In [ ]:
cnn3.add(tf.keras.layers.Dense(units=4, activation='softmax'))

## Part 3 - Training the CNN

In [ ]:
cnn3.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
cnn3.fit_generator(
    train_data,
    steps_per_epoch = train_data.samples // 64,
    validation_data = validation_data, 
    validation_steps = validation_data.samples // 64,
    epochs = 50)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
102/102 [==============================] - 2453s 24s/step - loss: 1.3383 - accuracy: 0.5146 - val_loss: 9.4670 - val_accuracy: 0.2494
Epoch 2/50
102/102 [==============================] - 33s 322ms/step - loss: 0.4840 - accuracy: 0.7683 - val_loss: 14.8662 - val_accuracy: 0.2463
Epoch 3/50
102/102 [==============================] - 33s 321ms/step - loss: 0.4509 - accuracy: 0.7926 - val_loss: 14.2072 - val_accuracy: 0.2500
Epoch 4/50
102/102 [==============================] - 33s 323ms/step - loss: 0.3856 - accuracy: 0.8286 - val_loss: 13.1431 - val_accuracy: 0.2469
Epoch 5/50
102/102 [==============================] - 33s 322ms/step - loss: 0.3653 - accuracy: 0.8418 - val_loss: 3.7946 - val_accuracy: 0.2631
Epoch 6/50
102/102 [==============================] - 33s 322ms/step - loss: 0.3479 - accuracy: 0.8519 - val_loss: 3.5986 - val_accuracy: 0.3056
Epoch 7/50
102/102 [==============================] - 33s 323ms/step - loss: 0.3418 - accuracy: 0.8511 - val_loss: 0.4956 - val

## Making a single prediction

In [ ]:
train_data.class_indices?

In [ ]:
from keras.preprocessing import image
def Pred(location):
  print(location)
  test_image = image.load_img(location, target_size = (64, 64))
  test_image = image.img_to_array(test_image)
  test_image = np.expand_dims(test_image, axis = 0)
  result = cnn3.predict(test_image)
  #train_data.class_indices?
  if(result[0][0] != 0):
      prediction = 'Empty Traffic'
  elif(result[0][1] != 0):
      prediction = 'High Traffic'
  elif(result[0][2] != 0):
      prediction = 'Low Traffic'      
  else:
    prediction = 'Medium Traffic'
  print(result)
  return prediction

In [ ]:
Pred('/content/drive/My Drive/lane1_4_parts/Low Traffic/frame_lane1__0.12.jpg')

/content/drive/My Drive/lane1_4_parts/Low Traffic/frame_lane1__0.12.jpg
[[0. 0. 0. 1.]]


'Medium Traffic'

In [ ]:
Pred('/content/drive/My Drive/lane1_4_parts/Empty Traffic/frame_lane0001_48.0.jpg')

/content/drive/My Drive/lane1_4_parts/Empty Traffic/frame_lane0001_48.0.jpg
[[0.0000000e+00 0.0000000e+00 1.1158172e-18 1.0000000e+00]]


'Low Traffic'

In [ ]:
Pred('/content/drive/My Drive/lane1_4_parts/High Traffic/frame_lane1__1.62.jpg')

/content/drive/My Drive/lane1_4_parts/High Traffic/frame_lane1__1.62.jpg
[[0. 0. 0. 1.]]


'Medium Traffic'

In [ ]:
Pred('/content/drive/My Drive/lane1_4_parts/High Traffic/frame_lane1__7.86.jpg')

/content/drive/My Drive/lane1_4_parts/High Traffic/frame_lane1__7.86.jpg
[[0. 1. 0. 0.]]


'High Traffic'

In [ ]:
Pred('/content/drive/My Drive/dataset/single_prediction/TH06-INDIA-TRAFFIC.jfif') # It's High Traffic

/content/drive/My Drive/dataset/single_prediction/TH06-INDIA-TRAFFIC.jfif
[[0. 1. 0. 0.]]


'High Traffic'

In [ ]:
Pred('/content/drive/My Drive/dataset/single_prediction/highway-with-high-traffic-KRC1PT.jpg')

/content/drive/My Drive/dataset/single_prediction/highway-with-low-traffic-KRC1PT.jpg
[[0. 1. 0. 0.]]


'High Traffic'

In [ ]:
cnn3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 48)        13872     
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 48)        192       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 48)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        27712     
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 64)        2